In [46]:
import os
import re

fairy_tales_folder = "../evaluation_data/baseline1/experiment_samit_fairy_tale_segmented"
short_story_folder = "../evaluation_data/baseline1/experiment_samit_short_story_segmented"

fairy_tales_names = []
short_stories_names = []

pattern = r"stories_(.*?)_"
for filename in os.listdir(fairy_tales_folder):
    if filename.startswith("df_images") or filename.startswith("df_summary"):
        continue
    match = re.search(pattern, filename)
    if match:
        story_name = match.group(1)
        fairy_tales_names.append(story_name)

# Remove duplicates and print the list
fairy_tales_names = list(set(fairy_tales_names))
print(fairy_tales_names)

for filename in os.listdir(short_story_folder):
    if filename.startswith("df_images") or filename.startswith("df_summary"):
        continue
    match = re.search(pattern, filename)
    if match:
        story_name = match.group(1)
        short_stories_names.append(story_name)

# Remove duplicates and print the list
short_stories_names = list(set(short_stories_names))
print(short_stories_names)


['king thrushbeard', 'little brother and little sister', 'clever hans', 'herr korbes', 'briar rose', 'clever elsie', 'faithful john', 'gods food', 'gambling hansel', 'hansel and gretel', 'how six men got on in the world', 'frau trude', 'cat and mouse in partnership', 'little snow white', 'fitchers bird', 'jorinda and joringel', 'going a travelling', 'frederick and catherine', 'knoist and his three sons', 'brother lustig', 'ferdinand the faithful', 'lean lisa', 'fundevogel bird foundling', 'domestic servants', 'hans married']
['the travelling companion', 'the frogs and the well', 'lost in a london fog', 'the rabbit the weasel and the cat', 'strength of the little thin thread', 'what the moon saw', 'the ass and its shadow', 'the candy country', 'a raven and a swan', 'mercury and the woodman', 'im a fool', 'the cat and the birds', 'the farmer and the snake', 'the man with the trumpet', 'the blind men and the elephant', 'the fox and the hedgehog', 'the lark and her young ones', 'the farmer

In [47]:
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import os
import pandas as pd

# Load CLIP model
device = "cuda:3" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

KeyboardInterrupt: 

In [ ]:
# Function to calculate CLIP score
def calculate_clip_score(text, image_path):
    print("Running calculate_clip_score")
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize([text]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        similarity = torch.cosine_similarity(text_features, image_features).cpu().numpy()

        # Get the logits representing the similarity score
        # logits = model(image_features, text_features).logits_per_image

    # return logits.item()
        # print(similarity)
        return similarity[0]

In [ ]:
# Function to process a story
def process_story(df, story_name, image_folder):
    print("Running process_story")
    story_df = df[df['topic'].str.contains(story_name)]
    # story_df = df[story_filter]
    scores = []

    for _, row in story_df.iterrows():
        # print("Checking in: ",row['text'])
        segment_no = row['topic'].split('_')[-1]

        print("segment_no: ", segment_no)
        image_filename1 = f"stories_{story_name}_{segment_no}.jpg"
        image_filename2 = f"stories_{story_name}_segment_{segment_no}.jpg"
        image_path1 = os.path.join(image_folder, image_filename1)
        image_path2 = os.path.join(image_folder, image_filename2)
        # print(image_path)
        if os.path.exists(image_path1):
          image_path = image_path1
        elif os.path.exists(image_path2):
          image_path = image_path2

        if os.path.exists(image_path):
            # print(image_path)
            # print(row['summary'])
            score = calculate_clip_score(row['summary'], image_path)
            scores.append(score)
            
    print(scores)
    return sum(scores) / len(scores) if scores else 0
    # return scores

In [89]:
# BASELINE 1 FAIRY TALES
image_folder = "../evaluation_data/baseline1/experiment_samit_fairy_tale_segmented"

df_images = pd.read_csv(image_folder+'/df_images.csv')
df_images.head()

fairy_tales_names_temp = ["briar rose", "cat and mouse in partnership","clever hans","faithful john","ferdinand the faithful","fitchers bird","gods food","hans married","herr korbes","how six men got on in the world","jorinda and joringel","lean lisa","little snow white", "hansel and gretel"]

average_scores_fairy_tales = {story: process_story(df_images, story, image_folder) for story in fairy_tales_names_temp}
print(average_scores_fairy_tales)

sums = 0
length = 0
for key,value in average_scores_fairy_tales.items():
    sums = sums+value
    length +=1
avg = sums/length
print(avg)

Running process_story
segment_no:  2.txt
Running calculate_clip_score
segment_no:  5.txt
Running calculate_clip_score
segment_no:  6.txt
Running calculate_clip_score
segment_no:  16.txt
Running calculate_clip_score
segment_no:  13.txt
Running calculate_clip_score
segment_no:  0.txt
Running calculate_clip_score
segment_no:  9.txt
Running calculate_clip_score
segment_no:  1.txt
Running calculate_clip_score
segment_no:  4.txt
Running calculate_clip_score
segment_no:  10.txt
Running calculate_clip_score
segment_no:  3.txt
Running calculate_clip_score
segment_no:  7.txt
Running calculate_clip_score
segment_no:  12.txt
Running calculate_clip_score
segment_no:  8.txt
Running calculate_clip_score
segment_no:  11.txt
Running calculate_clip_score
segment_no:  14.txt
Running calculate_clip_score
segment_no:  15.txt
Running calculate_clip_score
[0.2764, 0.2817, 0.2942, 0.2935, 0.2659, 0.259, 0.313, 0.2925, 0.258, 0.2556, 0.2279, 0.2598, 0.267, 0.3062, 0.2822, 0.2974, 0.307]
Running process_story
s

In [ ]:
# BASELINE 1 SHORT STORY
image_folder = "../evaluation_data/baseline1/experiment_samit_short_story_segmented"

df_images = pd.read_csv(image_folder+'/df_images.csv')
df_images.head()

average_scores_short_stories = {story: process_story(df_images, story, image_folder) for story in short_stories_names}
print(average_scores_short_stories)

sums = 0
length = 0
for key,value in average_scores_short_stories.items():
    sums = sums+value
    length +=1
avg = sums/length
print(avg)

Running process_story
segment_no:  12.txt
Running calculate_clip_score
segment_no:  4.txt
Running calculate_clip_score
segment_no:  33.txt
Running calculate_clip_score
segment_no:  34.txt
Running calculate_clip_score
segment_no:  20.txt
Running calculate_clip_score
segment_no:  0.txt
Running calculate_clip_score
segment_no:  53.txt
Running calculate_clip_score
segment_no:  26.txt
Running calculate_clip_score
segment_no:  10.txt
Running calculate_clip_score
segment_no:  1.txt
Running calculate_clip_score
segment_no:  22.txt
Running calculate_clip_score
segment_no:  45.txt
Running calculate_clip_score
segment_no:  17.txt
Running calculate_clip_score
segment_no:  41.txt
Running calculate_clip_score
segment_no:  3.txt
Running calculate_clip_score
segment_no:  43.txt
Running calculate_clip_score
segment_no:  36.txt
Running calculate_clip_score
segment_no:  52.txt
Running calculate_clip_score
segment_no:  19.txt
Running calculate_clip_score
segment_no:  8.txt
Running calculate_clip_score
seg

In [91]:
# Function to process a story
import json
import sys
def process_story_baseline2(df, story_name, image_folder):
    print("Running process_story", story_name)
    row = df[df["story_name"] == story_name].iloc[0]

    # print("selected story", df_temp.head())
    story_summaries = eval(row["summarized_story"])

    scores = []

    for segment_no, summary in enumerate(story_summaries):
        try:
          print("segment_no: ", segment_no)
          image_filename1 = f"{story_name}_{segment_no}.jpg"
          image_filename2 = f"{story_name}_segment_{segment_no}.jpg"
          image_path1 = os.path.join(image_folder, image_filename1)
          image_path2 = os.path.join(image_folder, image_filename2)
          
          
          print(image_path1)
          print(image_path2)
          if os.path.exists(image_path1):
            image_path = image_path1
          elif os.path.exists(image_path2):
            image_path = image_path2

          if os.path.exists(image_path):
              # print(image_path)
              # print(row['summary'])
              score = calculate_clip_score(summary, image_path)
              scores.append(score)
        except:
          print("error here")

    return sum(scores) / len(scores) if scores else 0
    # return scores

In [88]:
# BASELINE 2 FAIRY TALES
image_folder = "../evaluation_data/baseline2/baseline2_fairytales"

df_images = pd.read_csv(image_folder+'/df_summary.csv', index_col=None)

fairy_tales_names_temp = ["briar rose", "cat and mouse in partnership","clever hans","faithful john","ferdinand the faithful","fitchers bird","gods food","hans married","herr korbes","how six men got on in the world","jorinda and joringel","lean lisa","little snow white", "hansel and gretel"]
average_scores_fairy_tales = {story: process_story_baseline2(df_images, story, image_folder) for story in fairy_tales_names_temp}
print(average_scores_fairy_tales)

sums = 0
length = 0
for key,value in average_scores_fairy_tales.items():
    sums = sums+value
    length +=1
avg = sums/length
print(avg)

Running process_story briar rose
segment_no:  0
../evaluation_data/baseline2/baseline2_fairytales/briar rose_0.jpg
../evaluation_data/baseline2/baseline2_fairytales/briar rose_segment_0.jpg
Running calculate_clip_score
segment_no:  1
../evaluation_data/baseline2/baseline2_fairytales/briar rose_1.jpg
../evaluation_data/baseline2/baseline2_fairytales/briar rose_segment_1.jpg
Running calculate_clip_score
segment_no:  2
../evaluation_data/baseline2/baseline2_fairytales/briar rose_2.jpg
../evaluation_data/baseline2/baseline2_fairytales/briar rose_segment_2.jpg
Running calculate_clip_score
segment_no:  3
../evaluation_data/baseline2/baseline2_fairytales/briar rose_3.jpg
../evaluation_data/baseline2/baseline2_fairytales/briar rose_segment_3.jpg
Running calculate_clip_score
segment_no:  4
../evaluation_data/baseline2/baseline2_fairytales/briar rose_4.jpg
../evaluation_data/baseline2/baseline2_fairytales/briar rose_segment_4.jpg
Running calculate_clip_score
segment_no:  5
../evaluation_data/bas

In [85]:
a = [" A tale of a queen's wish to have a child as white as snow and as black", ' A little girl was born after the death of the Queen of England.', ' A young girl was born in a kingdom in the Middle East, and she grew up to', ' The Queen of England was looking through the looking-glass at the time of her coronation', 'Snow-white is the fairest of all, according to the story of the fairyt', ' A young girl called Snow-white came into the room of the Queen, and spoke to', 'Snow-white was a little girl, and she grew up to be a princess, and', 'Snow-white was a little girl who lived in the great forest of Snow-white,', ' A little girl was running along a path in the middle of a field in the country of', ' A little Snow-white was taken to a palace in the centre of the world, and', ' A small cottage in the hills of northern Ireland is home to seven dwarfs.', ' A group of people have been asking each other what they have been eating.', ' Seven dwarf children slept together in their bed, all night, and all seven slept together.', ' A little girl called Snow-white came to the home of seven dwarfs, who had', "Snow-white's heart was eaten by the Queen, but the Queen still thought she was", ' A tale of Snow-white and the seven dwarfs is told by a young girl in', "Snow-white's mother, the Queen of the looking-glass, said 'This is", ' The Queen of the Seven dwarfs wanted to kill Snow-white, because she was not', ' A tale of Snow-white and the Seven dwarfs, a little girl, and the', ' A fairytale is written by a wicked woman, who went to see Snow-white', ' An old woman, whose name is Snow-white, went to the seven dwarfs to', "Snow-white's step-mother poisoned her to death with a poisoned comb, and Snow", " The Queen's great-grandfather, Queen Elizabeth II, visited a wall of glass in", " A glass of gold opened up at the entrance of the Snow-white's castle, and", ' A little girl in the country of Pembrokeshire, in the south of', ' A young girl called Snow-white was so hungry for an apple that she went to the', ' An old woman called Snow-white came to the aid of an old woman who wanted to', " A selection of quotes from Shakespeare's Hamlet:", " The Snow-white's heart was so envious that it took three days for it to", ' A thousand years ago, Snow-white died, and she was still alive.', 'And then Snow-white died, and she was buried.', ' A thousand years ago Snow-white was buried in a box of gold, in a forest', 'Snow-white was buried in a coffin, with a piece of apple in her throat,', ' The Prince of Wales asked Snow-white to marry him, and asked her to go to', ' A look back at some of the most famous songs of the 20th Century.', ' A wicked woman called Snow-white was forced to dance with red-hot slippers,', "Snow White is one of the most popular stories in the BBC Children's Book."]


In [92]:
# BASELINE 2 SHORT STORY
image_folder = "../evaluation_data/baseline2/baseline2_short_stories"

df_images = pd.read_csv(image_folder+'/df_summary.csv', index_col=None)

average_scores_fairy_tales = {story: process_story_baseline2(df_images, story, image_folder) for story in short_stories_names}
print(average_scores_fairy_tales)

sums = 0
length = 0
for key,value in average_scores_fairy_tales.items():
    sums = sums+value
    length +=1
avg = sums/length
print(avg)

Running process_story the travelling companion
segment_no:  0
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_0.jpg
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_segment_0.jpg
Running calculate_clip_score
segment_no:  1
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_1.jpg
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_segment_1.jpg
Running calculate_clip_score
segment_no:  2
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_2.jpg
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_segment_2.jpg
Running calculate_clip_score
segment_no:  3
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_3.jpg
../evaluation_data/baseline2/baseline2_short_stories/the travelling companion_segment_3.jpg
Running calculate_clip_score
segment_no:  4
../evaluation_data/baseline2/baseline2_short_stories/the t